In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import requests
import time

from citipy import citipy
from config import weather_api_key
from datetime import datetime


In [2]:
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [3]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

777

In [4]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [5]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
       
        current_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Current Description": current_description,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | rikitea
Processing Record 2 of Set 1 | saint george
Processing Record 3 of Set 1 | tsihombe
City not found. Skipping...
Processing Record 4 of Set 1 | shamkhal
Processing Record 5 of Set 1 | hobart
Processing Record 6 of Set 1 | busselton
Processing Record 7 of Set 1 | rocha
Processing Record 8 of Set 1 | guerrero negro
Processing Record 9 of Set 1 | ushuaia
Processing Record 10 of Set 1 | severo-yeniseyskiy
Processing Record 11 of Set 1 | amderma
City not found. Skipping...
Processing Record 12 of Set 1 | bredasdorp
Processing Record 13 of Set 1 | jever
Processing Record 14 of Set 1 | albany
Processing Record 15 of Set 1 | beringovskiy
Processing Record 16 of Set 1 | faanui
Processing Record 17 of Set 1 | sarlat-la-caneda
Processing Record 18 of Set 1 | madingou
Processing Record 19 of Set 1 | kaeo
Processing Record 20 of Set 1 | umzimvubu
City not found. Skipping...
Processing Record 21 of Set 

In [6]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,light rain,72.86,76,100,25.97,PF,2022-09-07 03:51:26
1,Saint George,37.1041,-113.5841,broken clouds,99.63,16,53,4.76,US,2022-09-07 03:56:23
2,Shamkhal,43.0596,47.3373,light rain,59.02,90,100,6.31,RU,2022-09-07 03:57:03
3,Hobart,-42.8794,147.3294,broken clouds,57.06,64,75,10.36,AU,2022-09-07 03:54:01
4,Busselton,-33.6500,115.3333,broken clouds,62.53,59,83,10.25,AU,2022-09-07 03:57:04
5,Rocha,-34.4833,-54.3333,scattered clouds,51.67,99,26,5.12,UY,2022-09-07 03:56:26
6,Guerrero Negro,27.9769,-114.0611,clear sky,81.09,72,5,4.50,MX,2022-09-07 03:54:18
7,Ushuaia,-54.8000,-68.3000,light snow,29.86,93,75,12.66,AR,2022-09-07 03:57:04
8,Severo-Yeniseyskiy,60.3725,93.0408,overcast clouds,46.98,72,99,1.66,RU,2022-09-07 03:57:05
9,Bredasdorp,-34.5322,20.0403,few clouds,44.28,90,21,2.44,ZA,2022-09-07 03:57:05


In [7]:
# reorder the DataFrame
new_column_order = ['City', 'Country', 'Date', 'Current Description', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed']
city_data_df = city_data_df[new_column_order]

# view DataFrame
city_data_df

,City,Country,Date,Current Description,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Rikitea,PF,2022-09-07 03:51:26,light rain,-23.1203,-134.9692,72.86,76,100,25.97
1,Saint George,US,2022-09-07 03:56:23,broken clouds,37.1041,-113.5841,99.63,16,53,4.76
2,Shamkhal,RU,2022-09-07 03:57:03,light rain,43.0596,47.3373,59.02,90,100,6.31
3,Hobart,AU,2022-09-07 03:54:01,broken clouds,-42.8794,147.3294,57.06,64,75,10.36
4,Busselton,AU,2022-09-07 03:57:04,broken clouds,-33.6500,115.3333,62.53,59,83,10.25
...,...,...,...,...,...,...,...,...,...,...
707,Sosnovo-Ozerskoye,RU,2022-09-07 04:16:34,overcast clouds,52.5249,111.5418,61.29,51,99,17.81
708,Henties Bay,NA,2022-09-07 04:16:34,clear sky,-22.1160,14.2845,56.05,79,0,6.60
709,Mahibadhoo,MV,2022-09-07 04:16:35,overcast clouds,3.7833,72.9667,79.86,81,91,13.56
710,Yuxia,CN,2022-09-07 04:16:35,overcast clouds,34.0615,108.6291,79.41,43,95,3.80


In [8]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")